# DES_PoR_Tool

Developed in 2018 by Cemre Çubukçuoğlu - BK <C.Cubukcuoglu@tudelft.nl> 

This is a Discrete-Event Simulation (DES) tool developed as a parametric CAD program for validating program of requirements (PoR) for hospital space planning. The DES model simulates the procedures of processing of patients treated by doctors, calculating patient throughput and patient waiting times, based on the number of doctors, patient arrivals, and treatment times. In addition, the tool is capable of defining space requirements by taking hospital design standards into account. Using this tool, what-if scenarios and assumptions on the PoR pertaining to space planning can be tested and/or validated. The tool is ultimately meant for reducing patient waiting times and/or increasing patient throughput by checking the match of the layout of a hospital with respect to its procedural operations. This tool is envisaged to grow into a toolkit providing a methodological framework for bringing Operations Research into Architectural Space Planning. The tool is implemented in Python for [Grasshopper3D](https://www.grasshopper3d.com/), a plugin of McNeel's [Rhino3D](https://www.rhino3d.com/) CAD software using the [SimPy library](https://simpy.readthedocs.io/en/latest/).

Graphical Abstract: 
![alt text][logo]

[logo]: https://github.com/CemreTUDelft/DES_PoR_Tool/blob/master/DES_GraphicalAbstract.jpg "Logo Title Text 2"

The procedure run by the tool can be described as the following:
```
Begin
	Put the simulation settings
	Create DES model
		Create patient arrivals
		Create treatment process using GH inputs
		Run the simulation
	Get the outputs of DES

	Calculate space requirements for each unit
		Formulations using GH inputs and hospital design standards
	Get the outputs of PoR
End
```

# Installation

Before running the following cells, the [SimPy package](https://anaconda.org/mutirri/simpy) must be installed: 

To install this package using conda package-manager run one of the following:
```
conda install -c mutirri simpy
conda install -c mutirri/label/all simpy
conda install -c mutirri/label/anaconda simpy
```
To install this package [using pip package-manager](https://simpy.readthedocs.io/en/latest/simpy_intro/installation.html):
```
pip install simpy
```



## Inputs

In [26]:
# sample inputs based on the PoR of a hospital in Izmir, Turkey (to be newly built hospital in Seljuk)
NUM_DOCTORS=2
TREATMENTTIME=5
T_INTER=7
OutpatientDept_Num=16
SingleBedRoom_Num=20
MultipleBedRoom_BedNum=40
SingleBed_DeliveryRoom_Num=12
MultipleBed_DeliveryRoom_BedNum=10
ICU_BedNum=10
Administrative_Num=20
#this is just a sum of some of the above
TotalBeds_Num=SingleBedRoom_Num+MultipleBedRoom_BedNum+SingleBed_DeliveryRoom_Num+MultipleBed_DeliveryRoom_BedNum+ICU_BedNum+Administrative_Num

## Procedure

In [27]:
import random
import simpy

#Area calculation of treatment rooms & waiting areas in each outpatient department
if NUM_DOCTORS == 1:
    Outpatient_Dept_Area = (16 * NUM_DOCTORS) + 12 #treatment rooms + waiting areas
elif NUM_DOCTORS == 2:
    Outpatient_Dept_Area = (16 * NUM_DOCTORS) + 24 #treatment rooms + waiting areas
elif NUM_DOCTORS > 2:
    Outpatient_Dept_Area = (16 * NUM_DOCTORS) + 24 + (5 * (NUM_DOCTORS - 2)) #treatment rooms + waiting areas


In [28]:
RANDOM_SEED = 42 #seed number of simulation
SIM_TIME = 480 #simulation time in minutes: 8 hours
data_wait = []
data_patientdisposed=[]

### create outpatient components

In [29]:
#create outpatient components
class Outpatient(object):

    def __init__(self, env, num_doctors, treatmenttime):
        self.env = env
        self.doctor = simpy.Resource(env, num_doctors)
        self.treatmenttime = treatmenttime

    def treat(self, patient):
        yield self.env.timeout(TREATMENTTIME)


### create patient arrivals

In [30]:
#create patient arrivals
def patient(env, name, pr):
    print('%s arrives at the outpatient department at %.2f.' % (name, env.now))
    arrivetime=env.now
    with pr.doctor.request() as request:
        yield request 
        print('%s enters the outpatient department at %.2f.' % (name, env.now))
        entertime=env.now
        yield env.process(pr.treat(name))
        print('%s leaves the outpatient department at %.2f.' % (name, env.now))
        print('%s waiting time %.2f.' % (name, entertime-arrivetime))
        data_wait.append(entertime-arrivetime)


### create treatment process

In [31]:
#create treatment process
def setup(env, num_doctors, treatmenttime, t_inter):
    outpatient_department = Outpatient(env, num_doctors, treatmenttime)
    #create 4 initial patients
    for i in range(4):
        env.process(patient(env, 'patient %d' % i, outpatient_department))
    #create more patients
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(patient(env, 'patient %d' % i, outpatient_department))
        data_patientdisposed.append(i)

### run the simulation 

In [13]:
#run the simulation 
random.seed(RANDOM_SEED)  
env = simpy.Environment()
env.process(setup(env, NUM_DOCTORS, TREATMENTTIME, T_INTER))
env.run(until=SIM_TIME)
#get the outputs of DES from a,b
a=data_patientdisposed #number of patients disposed (treated) from the system
b=data_wait #number of waiting times

patient 0 arrives at the outpatient department at 0.00.
patient 1 arrives at the outpatient department at 0.00.
patient 2 arrives at the outpatient department at 0.00.
patient 3 arrives at the outpatient department at 0.00.
patient 0 enters the outpatient department at 0.00.
patient 1 enters the outpatient department at 0.00.
patient 4 arrives at the outpatient department at 5.00.
patient 0 leaves the outpatient department at 5.00.
patient 0 waiting time 0.00.
patient 1 leaves the outpatient department at 5.00.
patient 1 waiting time 0.00.
patient 2 enters the outpatient department at 5.00.
patient 3 enters the outpatient department at 5.00.
patient 5 arrives at the outpatient department at 10.00.
patient 2 leaves the outpatient department at 10.00.
patient 2 waiting time 5.00.
patient 3 leaves the outpatient department at 10.00.
patient 3 waiting time 5.00.
patient 4 enters the outpatient department at 10.00.
patient 5 enters the outpatient department at 10.00.
patient 4 leaves the ou

### calculate space requirements of hospital units

In [25]:
#calculate space requirements of hospital units
c=Outpatient_Dept_Area*OutpatientDept_Num #total area of all outpatient dept.s
d=(SingleBedRoom_Num*9)+(MultipleBedRoom_BedNum*7) #One-bed patient rooms must be min 9 m2.Patient wards must be min 7 m2 per each bed.
e=(SingleBed_DeliveryRoom_Num*12)+(MultipleBed_DeliveryRoom_BedNum*10) #One-bed delivery patient rooms must be min 12 m2.Delivery patient wards must be min 10 m2 per each bed.
f=ICU_BedNum*12 #ICU units must be min 12 m2 per each bed.
g=Administrative_Num*random.randint(8,12) #Administrative offices must be 8-12 m2 for each personnel. 
h=TotalBeds_Num+(TotalBeds_Num*0.2) #Bunker area = (number of beds) + (number of beds*20%)
#There must be min 6 elevators in 60-200 bed hospitals. 
#There must be min 9 elevators in 201-350 bed hospitals. 
if TotalBeds_Num >= 60 and TotalBeds_Num <= 200:
    MIN_ELEV = 6
elif TotalBeds_Num > 200 and TotalBeds_Num <= 350:
    MIN_ELEV = 9
i=MIN_ELEV
#get the outputs of PoR from c,d,e,f,g,h,i

## Output 

In [24]:
print('total area of all outpatient dept.s is: '+str(c))
print('Required area (m2) for inpatient departments is: '+str(d))
print('Required area (m2) for delivery is: '+str(e))
print('Required area (m2) for ICU is: '+str(f))
print('Required area (m2) for administration is: '+str(g))
print('Required area (m2) for bunker is: '+str(h))
# dimensions of a 2000 kg elevator
elevatorArea=1.5*2.7
print('minimum number of elevators is: '+str(i))
minElevArea=i*elevatorArea
print('thus, assuming elevators of size 1.5x2.7, the minimum area required for elevators is: '+str(minElevArea))


total area of all outpatient dept.s is: 896
Required area (m2) for inpatient departments is: 460
Required area (m2) for delivery is: 244
Required area (m2) for ICU is: 120
Required area (m2) for administration is: 180
Required area (m2) for bunker is: 134.4
minimum number of elevators is: 6
thus, assuming elevators of size 1.5x2.7, the minimum area required for elevators is: 24.300000000000004
